In [56]:
import minsearch
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables from .env
load_dotenv()

api_key = os.getenv("GOOGLE_GEMINI_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")



In [57]:
import json
with open('documents.json') as obj:
    raw_doc=json.load(obj)

In [58]:
documents=[];
for course_info in raw_doc: 
    for doc in course_info['documents']:
        doc['course']=course_info['course']
        documents.append(doc);

In [59]:
index=minsearch.Index(text_fields=['question','text','section'],keyword_fields=['course'])


In [60]:
index.fit(documents)

In [66]:
def search(query):
    boost={'question':3, 'section':0.4}
    results =index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results


In [72]:

def llm(prompt):
    response = model.generate_content(prompt)
    return response.text;

In [63]:


def build_prompt(query,search_results): 

    prompt_template="""  
    You are an course instructor assistant. Please read the question and give answer based on the context given. If no answer matched then say something like No answer found
    Also add a bit more details while answering the question.
    QUSETION: {question}
    CONTEXT:{context} 
    """.strip()
    
    context="" 
    
    for info in search_results:
        context=context+ f"section: {info['section']}\n question: {info['question']}\n answer: {info['text']}\n\n"; 
    
    final_prompt=prompt_template.format(question=query,context=context).strip()
    return final_prompt;


In [71]:
def rag(query): 
    retrive_data=search(query)
    prompt_with_context=build_prompt(query,retrive_data)
    # return prompt_with_context
    response=llm(prompt_with_context)
    return response;

In [73]:
rag("Can I still join the course?")

"Yes, you can still join the course even after the start date.  The provided text explicitly states that even if you don't register before the start date (January 15th, 2024, at 5 PM), you are still eligible to submit homework assignments.\n\nHowever, it's crucial to be aware of the deadlines for final projects to avoid last-minute stress.  While you can participate and submit homework,  procrastination on the final project is strongly discouraged.\n"